### Get Stop Words ###

In [ ]:
from glass3.dt.txtcls.san import get_stop_words

intbl  = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_ref\ref_fire_facebook_2017_2018.xlsx'
outtbl = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_ref\test_fire_face_1718.xlsx'

conParam = {
    'HOST': 'localhost', 'PORT' : '5432', 'PASSWORD' : 'admin',
    'USER' : 'postgres', 'TEMPLATE' : 'postgis_template'
}

get_stop_words(
    conParam, intbl, 'post_id', 'message', outtbl,
    inSheet='test'
)

### Go Machine Learning ###

In [ ]:
trainData = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_ref\train_fire_face_1718.xlsx'
classData = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_ref\test_fire_face_1718.xlsx'

trainSheet = 'train'
trainRefCol  = 'e_incendio'
trainDataCol = 'clean_message'
classDataCol = 'clean_message'

lang = None

outCorrWords = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_res\fire_corr_words.xlsx'
outPredict   = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_res\fire_predict.xlsx'
outEval      = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_res\fire_model_eval.xlsx'
outMxt       = '/home/jasp/todo/geotm/text_cls/tbl_res/mxt_svm.xlsx'

In [ ]:
from glass3.dt.txtcls.freq import correlated_words

correlated_words(
    trainData, trainRefCol, trainDataCol, outCorrWords,
    lang=lang, N=3, refSheet=trainSheet
)

In [ ]:
from glass3.dt.txtcls.cls import model_selection

model_selection(trainData, trainRefCol, trainDataCol, outEval, lang=lang)

In [ ]:
from glass3.dt.txtcls.cls import text_prediction
import os

methods = {
    'svm' : 'LinearSupportVectorMachine',
    'logistic' : 'LogisticRegression',
    'rf'       : 'RandomForest',
    'bayes'    : 'NaiveBayes'
}

for m in methods:
    name, ext = os.path.splitext(os.path.basename(outPredict))
    out = os.path.join(
        os.path.dirname(outPredict), "{}_{}{}".format(name, m, ext)
    )
    text_prediction(
        trainData, classData, trainRefCol, trainDataCol,
        classDataCol, out, method=methods[m]
    )

### SQL Based Methodology ###

In [ ]:
conParam = {
    "HOST" : "localhost", "PORT" : "5432", "PASSWORD" : "admin",
    "USER" : "postgres" , "DATABASE" : "dsn_collect"
}

TBL_SCHEMA = {
    "TNAME"   : "facedata",
    "TEXTCOL" : "lower_message",
    "TIMECOL" : "datahora",
    "SELCOL"  : ['post_id AS fid', 'type AS tipo', 'message AS mess'],
    "TEXTCASE" : (
        "CASE WHEN type = 'link' THEN lower(unaccent(description)) "
        "ELSE lower(unaccent(message)) END"
    )
}

START = '2017-06-01 00:00:00'
END   = '2018-10-31 23:59:59'

WORDS   = ['incendi', 'fog']

RESULT = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_res\sql_incendio1718.xlsx'

from glass3.dt.txtcls.qbase import get_rows_related_with_event

get_rows_related_with_event(
    conParam, TBL_SCHEMA, WORDS, RESULT,
    startTime=START, endTime=END
)

### Classification Evaluation ###

##### Binary classification evaluation ######

In [ ]:
from glass3.dt.txtcls.eval import binary_eval

ref_data = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_res\fire_predict_bayes.xlsx'
tst_data = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_res\fire_predict_bayes.xlsx'
eval_tbl = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_res\fire_bayes_valida.xlsx'

refIDCol = 'post_id'
refCol   = 'is_incendio_v2'
tstIDCol = 'post_id'
tstCol   = 'class_bin'

a = binary_eval(
    ref_data, refIDCol, refCol, tst_data, tstIDCol, eval_tbl, tstCol=tstCol
)

##### Multi-class classification evaluation #####

In [ ]:
from glass3.dt.txtcls.eval import model_conf_matrix

tst_data = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_res\fire_predict_bayes.xlsx'
outMxt   = r'D:\indev\expvgi\geotmlnh_text_cls\tbl_res\fire_bayes_valida2.xlsx'

refCol   = 'e_incendio'
tstCol   = 'classification'

model_conf_matrix(tst_data, refCol, tstCol, outMxt)